In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
# Log-ratio transformations
def clr_inv(p):
    z = tf.math.log(p)
    return z - tf.reduce_mean(z, axis=1)[:, tf.newaxis]

def clr_forward(z, axis=1):
    return tf.nn.softmax(z, axis=axis)

In [3]:
def wide_resnet(num_classes, depth=28, width=2):
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(16, (3, 3), padding="same")(inputs)
    for _ in range(depth // 6):
        x = layers.Conv2D(16 * width, (3, 3), padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)  # Use Keras's ReLU layer
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes)(x)
    return models.Model(inputs, outputs)

In [4]:
# Dataset loader combining TF CIFAR datasets with custom labels
def load_combined_dataset(cifar_version, label_file, num_classes):
    """
    Combine CIFAR image data from TensorFlow with labels from a .npy file.

    Args:
        cifar_version (str): "cifar10" or "cifar100" to select the dataset.
        label_file (str): Path to the .npy file containing labels.
        num_classes (int): Number of classes (10 or 100).

    Returns:
        tf.data.Dataset: Dataset ready for training/testing.
    """
    # Load image data
    if cifar_version == "cifar10":
        (x_train, _), _ = tf.keras.datasets.cifar10.load_data()
    elif cifar_version == "cifar100":
        (x_train, _), _ = tf.keras.datasets.cifar100.load_data()
    else:
        raise ValueError("cifar_version must be 'cifar10' or 'cifar100'.")

    # Load labels from .npy file
    label_data = np.load(label_file, allow_pickle=True).item()
    if 'clean_label' not in label_data:
        raise ValueError("Label file must contain 'clean_label' key.")

    # Normalize image data and convert labels
    x_train = x_train.astype(np.float32) / 255.0
    y_train = tf.keras.utils.to_categorical(label_data['clean_label'], num_classes)

    # Create TensorFlow dataset
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_ds = train_ds.shuffle(10000).batch(64)
    return train_ds

In [5]:
# Gaussian noise (manual implementation)
def add_gaussian_noise(inputs, mean=0.0, stddev=0.1):
    noise = tf.random.normal(shape=tf.shape(inputs), mean=mean, stddev=stddev)
    return inputs + noise

In [6]:
# Training step
@tf.function
def train_step(model, images, labels, optimizer):
    with tf.GradientTape() as tape:
        logits = model(images, training=True)
        smoothed_targets = clr_forward(labels)
        logit_targets = clr_inv(smoothed_targets)
        loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(logit_targets, logits))
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [ ]:
import pandas as pd

def train_model(train_ds, epochs=10, num_classes=10):
    model = wide_resnet(num_classes)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Define metrics
    accuracy_metric = tf.keras.metrics.CategoricalAccuracy()
    precision_metric = tf.keras.metrics.Precision()
    recall_metric = tf.keras.metrics.Recall()

    # DataFrame to store metrics
    metrics_df = pd.DataFrame(columns=["Epoch", "Loss", "Accuracy", "Precision", "Recall", "F1-Score"])

    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")
        epoch_loss_sum = 0
        num_steps = 0

        # Reset metrics at the start of each epoch
        accuracy_metric.reset_state()
        precision_metric.reset_state()
        recall_metric.reset_state()

        for step, (images, labels) in enumerate(train_ds):
            # Perform training step
            loss = train_step(model, images, labels, optimizer)
            
            # Update metrics
            epoch_loss_sum += loss.numpy()
            num_steps += 1
            predictions = model(images, training=False)
            accuracy_metric.update_state(labels, predictions)
            precision_metric.update_state(labels, predictions)
            recall_metric.update_state(labels, predictions)

            if step % 10 == 0:
                print(f"Step {step}, Loss: {loss.numpy()}")

        # Compute metrics for the epoch
        avg_loss = epoch_loss_sum / num_steps
        epoch_accuracy = accuracy_metric.result().numpy()
        epoch_precision = precision_metric.result().numpy()
        epoch_recall = recall_metric.result().numpy()
        epoch_f1_score = 2 * (epoch_precision * epoch_recall) / (epoch_precision + epoch_recall + 1e-7)

        # Display metrics
        print(f"Epoch {epoch + 1} - Average Loss: {avg_loss:.4f}, "
              f"Accuracy: {epoch_accuracy:.4f}, "
              f"Precision: {epoch_precision:.4f}, "
              f"Recall: {epoch_recall:.4f}, "
              f"F1-Score: {epoch_f1_score:.4f}")

        # Save metrics to DataFrame
        metrics_df = pd.concat([
            metrics_df,
            pd.DataFrame([{
                "Epoch": epoch + 1,
                "Loss": avg_loss,
                "Accuracy": epoch_accuracy,
                "Precision": epoch_precision,
                "Recall": epoch_recall,
                "F1-Score": epoch_f1_score
            }])
        ], ignore_index=True)

        # Save checkpoint with proper extension
        os.makedirs("checkpoints", exist_ok=True)
        model.save_weights(f"checkpoints/sgn_epoch_{epoch}.weights.h5")
        print(f"Checkpoint saved for epoch {epoch + 1}")

    # Save metrics to a CSV file
    metrics_df.to_csv("training_metrics.csv", index=False)
    print("Metrics saved to 'training_metrics.csv'")

In [8]:
if __name__ == "__main__":
    # Paths to label files
    cifar10_label_file = "./data/CIFAR-10_human_ordered.npy"
    cifar100_label_file = "./data/CIFAR-100_human_ordered.npy"

    # Load CIFAR-10 dataset for training
    train_ds_cifar10 = load_combined_dataset("cifar10", cifar10_label_file, num_classes=10)

    # Train on CIFAR-10 (CIFAR-100 can be loaded similarly)
    train_model(train_ds_cifar10, epochs=10, num_classes=10)


Epoch 1/10
Step 0, Loss: 1.6486167907714844
Step 10, Loss: 0.4435200095176697
Step 20, Loss: -1.1426292657852173
Step 30, Loss: -0.3364371657371521
Step 40, Loss: -0.4337804317474365
Step 50, Loss: -0.19734880328178406
Step 60, Loss: 2.5535049438476562
Step 70, Loss: 0.7717896103858948
Step 80, Loss: 0.411693811416626
Step 90, Loss: -0.1919950395822525
Step 100, Loss: 0.1359441876411438
Step 110, Loss: -1.8039202690124512
Step 120, Loss: 1.0233824253082275
Step 130, Loss: 0.4591432213783264
Step 140, Loss: -0.15268312394618988
Step 150, Loss: 0.6802430748939514
Step 160, Loss: 0.05132439732551575
Step 170, Loss: 0.2686876654624939
Step 180, Loss: 1.0770695209503174
Step 190, Loss: 0.7185894846916199
Step 200, Loss: 1.5800483226776123
Step 210, Loss: 0.10330504179000854
Step 220, Loss: -3.2299559116363525
Step 230, Loss: 0.8274235725402832
Step 240, Loss: 0.21352875232696533
Step 250, Loss: -0.8823997378349304
Step 260, Loss: 0.22993344068527222
Step 270, Loss: 0.5585253238677979
Step 

C:\Users\sbhatia2\AppData\Local\Temp\ipykernel_33536\982803084.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_df = pd.concat([


Step 10, Loss: -0.8859335780143738
Step 20, Loss: -1.0243818759918213
Step 30, Loss: 0.46508583426475525
Step 40, Loss: 0.8832007646560669
Step 50, Loss: -2.257573366165161
Step 60, Loss: -0.12125816941261292
Step 70, Loss: 0.00027620792388916016
Step 80, Loss: 0.3085614740848541
Step 90, Loss: -0.41359877586364746
Step 100, Loss: 0.16238999366760254
Step 110, Loss: -1.3019776344299316
Step 120, Loss: -0.5895940661430359
Step 130, Loss: 2.0273990631103516
Step 140, Loss: -0.0037988126277923584
Step 150, Loss: -0.30946171283721924
Step 160, Loss: 0.42292946577072144
Step 170, Loss: 0.4401721954345703
Step 180, Loss: -0.16463184356689453
Step 190, Loss: -0.4752507209777832
Step 200, Loss: 0.3126112222671509
Step 210, Loss: 1.0764703750610352
Step 220, Loss: 2.3178954124450684
Step 230, Loss: -0.12880921363830566
Step 240, Loss: 1.8546595573425293
Step 250, Loss: 0.38878583908081055
Step 260, Loss: -1.050205945968628
Step 270, Loss: 0.33870929479599
Step 280, Loss: -0.688868522644043
Step